In [7]:
import pandas as pd
import numpy as np
import shapely
import folium
from folium.plugins import HeatMap

In [8]:
country_data = pd.read_csv('./country_area/API_AG.LND.TOTL.K2_DS2_en_csv_v2_1036.csv')
city_data = pd.read_csv('./cities/worldcities.csv')
continent_data = pd.read_csv('./Countries-by-continents.csv')

In [9]:
# For each country, pick the latest value of the area
country_data = country_data.dropna(subset=['2022'])
country_data = country_data[['Country Name', '2022']]

# Drop countries that don't appear in the city data
country_data = country_data[country_data['Country Name'].isin(city_data['country'])]
country_data['Continent'] = country_data['Country Name'].map(continent_data.set_index('Country')['Continent'])
country_data['Continent'].dropna(inplace=True)
country_data.rename(columns={'Country Name': 'country', '2022': 'area'}, inplace=True)

city_data.dropna(subset=['population'], inplace=True)

In [10]:
def polygon_random_points (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
        random_point = shapely.Point([np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points

In [13]:
def generate_country_data():
    # We first sample a continent with equal probability
    continent = np.random.choice(country_data['Continent'].unique())
    continent_countries = country_data[country_data['Continent'] == continent]
    if continent_countries.empty:
        return generate_country_data()
    # We first sample a country with a probability proportional to its surface area compared to the Earth’s total surface area.
    country = np.random.choice(continent_countries['country'], p=continent_countries['area']/continent_countries['area'].sum())
    # Randomly sample a city from the selected country
    picked_city_data = city_data[city_data['country'] == country]
    if picked_city_data.empty:
        return generate_country_data()
    # Randomly sample a city based on the population
    city_name = np.random.choice(picked_city_data['city'], p=picked_city_data['population']/picked_city_data['population'].sum())
    city = city_data[city_data['city'] == city_name].iloc[0]

    city_point = shapely.geometry.Point(city['lng'], city['lat'])
    city_buffer = city_point.buffer(0.25)

    return (country, city, polygon_random_points(city_buffer, 100))

In [14]:
m = folium.Map(zoom_start=2.2)

for i in range(100):
    country, city, points = generate_country_data()
    # folium.Marker([city['lat'], city['lng']], popup=city['city']).add_to(m)
    # for p in points:
    #     folium.Marker([p.y, p.x], icon=folium.Icon(color='green')).add_to(m)
    HeatMap([[p.y, p.x] for p in points], radius=20, blur=30).add_to(m)

m

In [25]:
country, city, points = generate_country_data()

('United States',
 city            Springfield
 city_ascii      Springfield
 lat                 42.1155
 lng                -72.5395
 country       United States
 iso2                     US
 iso3                    USA
 admin_name    Massachusetts
 capital                 NaN
 population         437752.0
 id               1840000466
 Name: 1655, dtype: object)

In [29]:
from GoogleDataDownloader import StreetViewer

sv = StreetViewer(verbose=False)
for p in points:
    sv.get_meta(f"{p.y},{p.x}")
    sv.get_pic(f"{p.y},{p.x}")

>>> Obtained Meta from StreetView API:
{'copyright': '© Google', 'date': '2019-08', 'location': {'lat': 41.98445813658476, 'lng': -72.53699156250781}, 'pano_id': '2Ec874ah2EfjEjyXqJ-j1w', 'status': 'OK'}
>>> Picture available, requesting now...
>>> Saving objects to ./streetviews/
>>> COMPLETE!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>> Obtained Meta from StreetView API:
{'status': 'ZERO_RESULTS'}
>>> Picture not available in StreetView, ABORTING!
>>